# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.

If you need to start a 'notebook' again, go to Kernel menu >> Restart kernel.

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect.

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder for more ideas!

## Business value of these exercises

A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. Also, double check you have the right kind of API token with the right permissions. You should find it on [this webpage](https://platform.openai.com/api-keys) and it should show with Permissions of "All". If not, try creating another key by:
- Pressing "Create new secret key" on the top right
- Select **Owned by:** you, **Project:** Default project, **Permissions:** All
- Click Create secret key, and use that new key in the code and the `.env` file (it might take a few minutes to activate)
- Do a Kernel >> Restart kernel, and execute the cells in this Jupyter lab starting at the top
4. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
5. See the [troubleshooting](troubleshooting.ipynb) notebook in this folder for more instructions
6. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [3]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

# Uncomment the below line if this gives you any problems:
# openai = OpenAI(api_key="your-key-here")

In [4]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        try:
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
            driver.get(url)
            
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            self.title = soup.title.string if soup.title else "No Title Found"
            for useless in soup.body(['script','style','img','input','noscript']):
                useless.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            driver.quit()

In [5]:
# Let's try one out

ed = Website("https://kiran.me.uk")
print(ed.url)
print(ed.title)
print(ed.text)

https://kiran.me.uk
KIRAN RAMBHA
Kiran Rambha
Hi, I'm Kiran Rambha
.
About
A full-stack web developer with over 7 years of experience, currently working with different JavaScript technologies like
NodeJS
,
ReactJS
and many other cloud services in
GCP
,
AWS
and
Azure
Stack. I also have experience in developing high-quality large scale web application using
ASP.NET CORE
,
C#
as well as developing Amazon Alexa skills using
Alexa Skill Kit
and
AWS Lambda
.
Along with being a full-stack developer, I really enjoy photography especially landscape and street photography. I recently started exploring the world of film photography and I love learning all the ins and outs of different film stocks and film cameras. Aside from photography I like to spend my free time at the gym or taking long walks at the local park. I also like to travel and explore food and culture from different countries.
Experience
Senior Software Engineer
Compare the Market
Specialized in:
Node Js, GraphqlQL, GitLab, React Js

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [11]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [7]:
def get_openai_message_format(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [8]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = get_openai_message_format(website)
    )
    return response.choices[0].message.content

In [26]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThis website is created by Ed Donner, a technology enthusiast with a focus on coding and large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company dedicated to using AI in talent discovery and management. Ed has a background as the founder and CEO of the AI startup untapt, which was acquired in 2021.\n\n## Key Features:\n\n- **Outsmart LLM Arena**: A unique platform that pits LLMs against each other in diplomatic and cunning challenges.\n  \n- **Blog Posts**: The website includes several informative posts, including:\n  - **From Software Engineer to AI Data Scientist – resources** (October 16, 2024)\n  - **Outsmart LLM Arena – a battle of diplomacy and deviousness** (August 6, 2024)\n  - **Choosing the Right LLM: Toolkit and Resources** (June 26, 2024)\n  - **Fine-tuning an LLM on your texts: a simulation of you** (February 7, 2024)\n\nEd invites visitors to connect and share interests in coding, LLMs, and electronic mus

In [9]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://kiran.me.uk")

# Kiran Rambha

## Overview
Kiran Rambha is a full-stack web developer with over 7 years of experience in JavaScript technologies, cloud services, and web application development. He specializes in NodeJS, ReactJS, and has a solid background in ASP.NET CORE and Alexa Skill development. Kiran is also passionate about photography, particularly landscape and street photography, and enjoys traveling, gym workouts, and exploring different cultures through food.

## Experience
Kiran has held various senior software engineering positions at notable companies:
- **Compare the Market** (Dec '22 - Present) - Focused on NodeJS, ReactJS, and microservices.
- **Profile Pensions** (Jul '21 - Nov '22) - Worked with NodeJS, ReactJS, and GCP.
- **Accenture UK** (Dec '19 - Jul '21) - Developed Alexa Skills among other technologies.

## Projects
1. **Stock Hub - Alexa Skill**: Provides stock price updates and company earnings reports.
2. **Kiran Rambha Website**: A personal site built with React and TailwindCSS, adhering to responsive design.
3. **Just Stream - Alexa Skill**: A search engine for finding streaming availability of movies on platforms like Netflix and Amazon Prime.
4. **Local Exchange Trading System**: A web application for exchanging goods and services with a built-in currency.

## Education
- **BSc Computer Science** from Royal Holloway, University of London (2013 - 2017)

## Skills
Kiran possesses a wide array of technical skills including NodeJS, React, AWS, Docker, and various programming languages.

## Additional Information
Kiran has expressed a growing interest in film photography and actively pursues learning about film stocks and cameras.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this.

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [29]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN is a prominent news outlet that provides a broad range of news coverage, including US and world news, politics, business, health, entertainment, sports, science, climate, and weather updates. The website features video content, live updates, and various analysis pieces on current events.

## Notable News Updates:
- **Israel-Iran Conflict**: The Israeli military has conducted strikes on air defense batteries in Iran amid escalating tensions. Analysts suggest the outcome may be influenced by the upcoming US elections.
  
- **Ukraine-Russia War**: Ongoing updates regarding the conflict, including its impact on humanitarian conditions.
  
- **Recent Domestic Events**: Michelle Obama criticizes Trump regarding his reactions and political maneuvers amid the ongoing war in Gaza. Additionally, significant elections are occurring in Georgia with allegations from opposition parties about electoral misconduct.

- **Global Incidents**: 
  - **Flooding in the Philippines**: Over 126 people are dead or missing due to severe flooding and landslides.
  - **Hoax Bomb Threats in India**: These incidents are causing chaos ahead of the Diwali festival.
  - **SpaceX**: An astronaut from the Crew-8 mission has been hospitalized post-splashdown, but is reported to be in stable condition.

- **Human Interest Stories**: Coverage includes a young climber who became the youngest person to summit the world’s highest peaks and heartwarming community initiatives.

The CNN website is a hub for staying informed on global and local events, along with engaging multimedia content and expert analyses.

In [30]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, dedicated to developing reliable and beneficial AI systems. The website showcases their flagship AI model, **Claude**, with the latest release being **Claude 3.5 Sonnet**, which is described as their most intelligent model. Claude is designed to enhance efficiency and create new revenue streams through its API.

## Recent Announcements
- **October 22, 2024**: Introduction of Claude 3.5 Sonnet and Claude 3.5 Haiku, along with updates to general computer use.
- **September 4, 2024**: Launch of **Claude for Enterprise**, aimed at providing tailored AI solutions for businesses.
- **December 15, 2022**: Research published on **Constitutional AI**, focusing on ensuring AI harmlessness through feedback.
- **March 8, 2023**: Release of insights on **AI Safety**, exploring essential considerations for the field.

The website also includes sections related to the company's team, research initiatives, career opportunities, and more detailed information on the Claude API and pricing.

In [86]:
display_summary("https://openai.com")

# OpenAI Website Summary

OpenAI is dedicated to creating safe artificial general intelligence (AGI) that benefits all of humanity. The website outlines various AI research advancements and product innovations while providing tools for users, teams, and enterprises to leverage AI technologies.

## Key Features:
- **OpenAI o1**: A new series of AI models focused on improving reasoning time before responding.
- **ChatGPT**: A versatile AI tool capable of various tasks such as planning, teaching, translating, and more.
- **Sora**: A video generation model designed to create realistic and imaginative video content from text input.
- **Canvas**: A new method for writing and coding with ChatGPT.
- **SearchGPT**: A prototype for new AI search features.

## Recent News & Announcements:
- **Partnership with Apple**: OpenAI and Apple announced a collaboration to integrate ChatGPT into Apple's platforms.
- **ChatGPT Enhancements**: The AI can now "see, hear, and speak," expanding its interactive capabilities.
- **GPT Store**: Introduction of a store feature that enhances the accessibility of different AI tools and models.

## Research Highlights:
- Studies focusing on improving model safety and reasoning capabilities are ongoing, with notable projects such as the development of an early warning system for potential AI misuse.

The site serves as a hub for users interested in accessing cutting-edge AI technologies while keeping abreast of ongoing research and product developments.

## Business Applications

In this exercise, you experienced calling the API of a Frontier Model (a leading model at the frontier of AI) for the first time. This is broadly applicable across Gen AI use cases and we will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them.

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c